#### Equal Weight Investing
This particular investment strategy is based on selecting a particular group of stocks and investing an equal amount of capital in each stock.
The BSE 500 is sort of the Indian equivalent of the S and P 500. It is market cap weighted. 

This script will accept portfolio values and calculate the ampount required to invest to create an equal weight investment

#### Library Imports 
Importing necessary libraries

In [27]:
import numpy as np
import pandas as pd
import xlsxwriter
from bsedata.bse import BSE
import math 

#### Importing the list of stocks

In [28]:
df = pd.read_csv("bse500_data.csv")
df.head()

,Security,ISIN,Sector,Industry,Date,Price,1 day change (%),52 Week Low,52 Week High,3 Year Low,3 Year High,5 Year Low,5 Year High,All Time Low,All Time High,Market Cap (Cr),Enterprise Value (Cr)
0,3M India Ltd.,INE470A01017,Diversified,Diversified,2022-12-06,"23,074.75",-0.15,"17,273.00","26,943.00","15,685.60","31,000.00","15,250.00","31,000.00",223.25,"31,000.00","25,963.68","24,677.77"
1,Aarti Drugs Ltd.,INE767A01016,Healthcare,Drugs & Pharma,2022-12-06,461.00,0.16,378.00,608.00,105.25,"1,026.95",104.83,"1,026.95",1.62,"1,026.95","4,287.38","4,776.07"
2,Aavas Financiers Ltd.,INE216P01012,Financial,Housing Finance,2022-12-06,"1,888.45",-0.09,"1,815.00","3,340.00",845.75,"3,340.00",NaN,NaN,611.50,"3,340.00","14,928.56","21,364.02"
3,ABB India Ltd.,INE117A01022,Capital Goods,Switching Equipment,2022-12-06,"2,962.00",-0.46,"1,944.60","3,446.30",722.00,"3,446.30",722.00,"3,446.30",29.27,"3,446.30","63,303.39","60,385.73"
4,Abbott India Ltd.,INE358A01014,Healthcare,Drugs & Pharma,2022-12-06,"20,099.80",0.20,"15,514.00","20,900.80","12,186.55","23,934.45","5,210.00","23,934.45",220.00,"23,934.45","42,602.94","39,873.16"


#### Extracting needed data from the csv file

In [29]:
cols_needed = ['Security','ISIN', 'Price', 'Market Cap (Cr)','Industry']
main_df = df[cols_needed]
main_df.head()



,Security,ISIN,Price,Market Cap (Cr),Industry
0,3M India Ltd.,INE470A01017,"23,074.75","25,963.68",Diversified
1,Aarti Drugs Ltd.,INE767A01016,461.00,"4,287.38",Drugs & Pharma
2,Aavas Financiers Ltd.,INE216P01012,"1,888.45","14,928.56",Housing Finance
3,ABB India Ltd.,INE117A01022,"2,962.00","63,303.39",Switching Equipment
4,Abbott India Ltd.,INE358A01014,"20,099.80","42,602.94",Drugs & Pharma


Checking all of the scrip codes for all the companies from the library. Converting it to an array that we will be parsing
through for more information on the given stocks 

In [30]:

b = BSE()
b = BSE(update_codes = True)
main_dict = b.getScripCodes()
scrip_codes = []


def get_key(val):
    for key, value in main_dict.items():
        if val == value:
            return key

    return "key doesn't exist"


for key,value in main_dict.items():
    for security in main_df:
        if (main_df[security] == value).any():
            scrip_codes.append(key)
        
print(scrip_codes)

['524348', '500488', '512599', '533096', '500003', '532331', '533573', '532749', '521070', '500008', '500425', '508869', '500877', '515030', '500477', '500820', '506820', '500027', '524804', '532215', '532977', '532978', '500490', '500038', '532134', '532149', '532525', '500043', '509480', '500049', '500493', '500103', '500547', '532454', '532523', '500335', '526612', '500067', '500530', '532929', '500825', '511196', '532483', '524742', '513375', '500870', '500878', '532885', '500040', '532443', '500087', '532210', '533278', '531344', '506395', '500480', '500096', '533309', '532848', '507717', '532488', '532922', '505200', '522074', '532178', '500133', '500086', '500144', '500940', '532809', '532843', '507815', '500660', '532296', '500163', '532424', '500164', '509488', '500300', '501455', '500173', '500670', '533248', '500690', '532702', '531531', '517354', '524735', '500440', '513599', '500104', '500696', '500188', '517174', '532814', '530965', '532388', '532612', '532514', '500209',

Inserting values into the new dataframe that consists of real time values

In [31]:
security_id = []
current_price = []
market_cap = []

try:
    for code in scrip_codes:
        quote = b.getQuote(code)
        security_id.append(quote['securityID'])
        current_price.append(quote['currentValue'])
        market_cap.append(quote['marketCapFull'])
except:
    print(f"Stock -> {quote['companyName']} is not traded on the BSE anymore")
    
active_df = pd.DataFrame()
active_df.insert(0, "Security", security_id)
active_df.insert(1, "Current Price", current_price)
active_df.insert(2, "Market Cap", market_cap)
active_df.insert(3, "Shares to be bought", np.nan)

active_df.head()


Stock -> DABUR INDIA LTD. is not traded on the BSE anymore


,Security,Current Price,Market Cap,Shares to be bought
0,AARTIDRUGS,457.05,"4,232.28 Cr.",NaN
1,ABBOTINDIA,20209.25,"42,943.25 Cr.",NaN
2,ADANIENT,3998.30,"4,55,806.65 Cr.",NaN
3,ADANIPOWER,324.35,"1,25,099.81 Cr.",NaN
4,AEGISLOG,336.70,"11,818.17 Cr.",NaN


Calculating the number of shares to be bought

In [32]:
portfolio_size = input('Enter the size of your portfolio')

try:
    portfolio_size_float = float(portfolio_size)
except ValueError:
    print("Please Enter a valid integer")
    print("Try again!")
    portfolio_size = input('Enter the size of your portfolio')
    portfolio_size_float = float(portfolio_size)
    

In [33]:
position = portfolio_size_float / len(active_df.index)

for i in range(len(active_df.index)):
    active_df.loc[i,'Shares to be bought'] = math.floor(position / float(active_df.loc[i,'Current Price']))
    
active_df.head()

,Security,Current Price,Market Cap,Shares to be bought
0,AARTIDRUGS,457.05,"4,232.28 Cr.",383.0
1,ABBOTINDIA,20209.25,"42,943.25 Cr.",8.0
2,ADANIENT,3998.30,"4,55,806.65 Cr.",43.0
3,ADANIPOWER,324.35,"1,25,099.81 Cr.",540.0
4,AEGISLOG,336.70,"11,818.17 Cr.",521.0


Converting final output to an Excel spreadsheet

In [34]:
excel_writer = pd.ExcelWriter('trading_recommendations.xlsx', engine = 'xlsxwriter')
active_df.to_excel(excel_writer, 'Trading Recommendations' , index = False) 


Formatting config for excel file

In [35]:
bg_col = '100E09'
font_col = 'ffffff'

format = excel_writer.book.add_format(
    {
        'font_color' : font_col,
        'bg_color' :  bg_col,
        'border' : 1
    }
)


rupee_format = excel_writer.book.add_format(
    {
        'num_format' : '₹0.00' , 
        'font_color' : font_col,
        'bg_color' :  bg_col,
        'border' : 1
    }
)

integer_format = excel_writer.book.add_format(
    {
        'num_format' : '0' ,
        'font_color' : font_col,
        'bg_color' :  bg_col,
        'border' : 1
    }
)

column_format = {
    'A' : ['Security', format],
    'B' : ['Current Price', rupee_format],
    'C' : ['Market Cap', rupee_format],
    'D' : ['Shares to be bought', integer_format],
}


Applying the formats and saving the excel sheet

In [36]:
for col in column_format:
    excel_writer.sheets['Trading Recommendations'].set_column(f'{col}:{col}', 25 , column_format[col][1])

excel_writer.save()

/tmp/ipykernel_144483/1196783018.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
